In [1]:
import requests
import datetime
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:
url = 'https://www.hasznaltauto.hu/talalatilista/PDNG2VG3R2RTADH5S56ADUDWM43H2XDJUV66SLZAO4YDJDMTKREUAU2G7X3XLAHNCKTE6WHH3A7L4BMQXNFH4BKZKZJGRCGYHGX6BBEDHAZFMSQRIY2EQ57IUON2BPIUIC46G2OBF7FDOGBRBTIEAGGF6EWIWJOJYAYIDL4NWIUUSIMF3AZPBOLGBZJGB37LFTXPJCSIMFHX2VVVICY35ARX7WKEPCTS7K7ETUMBA752DACUPDQVBJKOQGOH3QNR4YMT3UQFUN5UCPZ3O676CGXBBJFULXCLZNC7ELH6GRXMDSRHW3VGSDP65ODQX5QYPKFCC2PMLCVSQJVV67WZY7YF37HDCTAZ6ANGVVNMM34ZGUOXEDIQIL36KHO7U4CF3JWAGB3OB7RIWPJ5TFWE6XFFK2BYU6JCTZS4XZYSUM3NWTAZSWS6EMT3HY23A57IOICSBAU3RPRAR4RAX7KHYVGT6CKSYZ53JJ3DLW53MTXWM6475T6WYH6E6HAQND7SQK3EHX73N566MXD6SGOIDN7LOTGWOYTA2NJU6GHQHPCT34MMBHG6Q5BKX6OF4GUPH66SMAYRSY5SUFTHC7BKR6OS6QNP6IMFBKXFLH3LWBSBICLV6THLQUGDTEQAWSQ22TZVKWNVQ34ZENJJAMN5PCHGS3VMSFFGC6AX4UFDO33D5LBZTYJYNW5G7DAVS53R3Z2VSFW6D7TUA66HKC34452AE2V6Q3P6AXYNH7EPHYPH6AOHAFVHS'

In [3]:
out = []
maxpage = int(BeautifulSoup(requests.get(url).content,'html5lib').find('ul',{'class':'pagination'})\
            .find('li',{'class':'last'}).text.strip())

In [4]:
maxpage

1074

In [ ]:
for i in tqdm(range(1,maxpage+1)):
    soup = BeautifulSoup(requests.get(url + '/page%d' % i).content,'html5lib')
    for car in soup.find_all('div',{'class':'talalati-sor'}):
        dic = {'nev':car.find('h3').text.strip(),
               'link':car.find('h3').find('a')['href'],
              'price':car.find('div',{'class':'vetelar'}).text.strip()}
        for aind,s in enumerate(car.find('div',{'class':'adatok'}).find_all('span')):
            dic['adat-%d' % aind] = s.text.strip()
        cimkelista = car.find('div',{'cimke-lista'})
        if type(cimkelista) != type(None):
            for cimke in cimkelista.find_all('span'):
                dic[cimke.text] = 1
        out.append(dic.copy())

In [ ]:
pd.DataFrame(out).drop_duplicates().to_csv('data/hasznaltauto-%s.csv' % str(datetime.date.today()),index=None)